# Amazon Bedrock AgentCore Runtime에서 MCP Server 호스팅 - AWS IAM Inbound Authentication

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime에서 MCP (Model Context Protocol) server를 호스팅하는 방법을 배웁니다. Amazon Bedrock AgentCore Python SDK를 사용하여 MCP tool을 Amazon Bedrock AgentCore와 호환되는 MCP server로 래핑합니다.

Amazon Bedrock AgentCore Python SDK는 MCP server 구현 세부 사항을 처리하므로 tool의 핵심 기능에 집중할 수 있습니다. 코드를 AgentCore 표준화된 MCP protocol 계약으로 변환하여 직접 통신할 수 있도록 합니다.

[MCP protocol](https://modelcontextprotocol.io/docs/getting-started/intro) 사양은 전통적으로 인증을 위해 OAuth token을 요구하지만, AgentCore runtime은 MCP server에 대한 인바운드 요청에 AWS IAM 자격 증명을 구성할 수 있는 기능을 제공하여 중요한 엔터프라이즈 요구 사항을 해결합니다.

### 튜토리얼 세부 정보

| 정보         | 세부 사항                                                   |
|:--------------------|:----------------------------------------------------------|
| 튜토리얼 유형       | Hosting Tools                                             |
| Tool 유형           | MCP server                                                |
| 튜토리얼 구성 요소 | AgentCore Runtime에서 MCP server 호스팅                  |
| 튜토리얼 분야   | Cross-vertical                                            |
| 예제 복잡도  | Easy                                                      |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK and MCP               |

### 튜토리얼 아키텍처

이 튜토리얼에서는 MCP server를 AgentCore runtime에 배포하는 방법을 설명합니다.

데모 목적으로 3개의 tool이 있는 간단한 MCP server를 사용합니다: `add_numbers`, `multiply_numbers`, `greet_user`

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* 커스텀 도구로 MCP server 생성
* MCP server 로컬 테스트
* Amazon Bedrock AgentCore Runtime에서 MCP server 호스팅
* 인증을 사용하여 배포된 MCP server 호출


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명 구성
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) 라이브러리
* 실행 중인 Docker daemon

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# AgentCore Runtime 관리를 위한 toolkit
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
# AWS 서비스 접근을 위한 boto3 session
from boto3.session import Session
from pathlib import Path
import os

In [ ]:
boto_session = Session()
region = boto_session.region_name

# AgentCore 제어를 위한 클라이언트
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
# Parameter Store 접근을 위한 클라이언트
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_iam"

## MCP (Model Context Protocol) 이해하기

MCP는 AI model이 외부 데이터와 tool에 안전하게 액세스할 수 있도록 하는 protocol입니다. 주요 개념:

* **Tools**: AI가 작업을 수행하기 위해 호출할 수 있는 함수
* **Streamable HTTP**: AgentCore Runtime에서 사용하는 전송 protocol
* **Session Isolation**: 각 클라이언트는 `Mcp-Session-Id` 헤더를 통해 격리된 세션을 얻습니다
* **Stateless Operation**: Server는 확장성을 위해 stateless 작동을 지원해야 합니다

AgentCore Runtime은 MCP server가 기본 경로로 `0.0.0.0:8000/mcp`에 호스팅될 것으로 예상합니다.

### 프로젝트 구조

적절한 구조로 프로젝트를 설정해 보겠습니다:

```
mcp_server_project/
├── mcp_server.py              # 메인 MCP server 코드
├── mcp_client.py          # 로컬 테스트 클라이언트
├── mcp_client_remote.py   # 원격 테스트 클라이언트
├── requirements.txt          # 의존성
└── __init__.py              # Python 패키지 마커
```

## MCP Server 생성

이제 세 가지 간단한 tool이 있는 MCP server를 생성해 보겠습니다. Server는 AgentCore Runtime 호환성을 위해 필요한 `stateless_http=True`와 함께 FastMCP를 사용합니다.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

# stateless_http=True: AgentCore Runtime 호환성을 위해 필수
mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    # streamable-http: AgentCore Runtime에서 사용하는 전송 protocol
    mcp.run(transport="streamable-http")

### 이 코드가 하는 일

* **FastMCP**: tool을 호스팅할 수 있는 MCP server를 생성합니다
* **@mcp.tool()**: Python 함수를 MCP tool로 변환하는 데코레이터
* **stateless_http=True**: AgentCore Runtime 호환성을 위해 필요
* **Tools**: 다양한 유형의 작업을 보여주는 세 가지 간단한 tool

## 로컬 테스트 클라이언트 생성

AgentCore Runtime에 배포하기 전에 MCP server를 로컬에서 테스트할 클라이언트를 생성해 보겠습니다:

In [ ]:
%%writefile mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    # terminate_on_close=False: 연결 종료 시 server를 종료하지 않음
    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

 ### 로컬 테스트

MCP server를 로컬에서 테스트하려면:

1. **터미널 1**: MCP server 시작
   ```bash
   python mcp_server.py
   ```
   
2. **터미널 2**: 테스트 클라이언트 실행
   ```bash
   python mcp_client.py
   ```

출력에서 세 가지 tool이 나열되는 것을 볼 수 있습니다.

## AgentCore Runtime Deployment 구성

다음으로 starter toolkit을 사용하여 entrypoint, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime deployment를 구성합니다. 또한 시작 시 Amazon ECR repository를 자동으로 생성하도록 starter kit을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
print(f"Using AWS region: {region}")

# 필수 파일 존재 확인
required_files = ["mcp_server.py", "requirements.txt"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
# auto_create_execution_role: IAM 실행 역할 자동 생성
# auto_create_ecr: ECR repository 자동 생성
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name=tool_name,
)
print("Configuration completed ✓")

## AgentCore Runtime에 MCP Server 시작

이제 docker 파일이 있으므로 MCP server를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR repository와 AgentCore Runtime이 생성됩니다

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

In [ ]:

# Agent ARN을 Parameter Store에 저장 (다른 스크립트에서 재사용 가능)
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime_iam/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server with inbound auth',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## 원격 테스트 클라이언트 생성

이제 배포된 MCP server를 테스트할 클라이언트를 생성해 보겠습니다. 이 클라이언트는 AWS에서 필요한 자격 증명을 검색하고 배포된 server에 연결합니다:

In [ ]:
%%writefile mcp_client_remote.py       
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
# AWS SigV4 인증을 지원하는 streamable HTTP client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # 현재 boto3 session에서 AWS 자격 증명 가져오기
    session = boto3.Session()
    credentials = session.get_credentials()

    # SigV4 서명 기능을 가진 custom transport 생성
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # 모든 페이지를 가져올 때까지 반복
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    # Parameter Store에서 Agent ARN 가져오기
    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    # URL encoding: ARN의 특수 문자를 URL 안전 형식으로 변환
    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
            mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, "inputSchema") and tool.inputSchema:
                        properties = tool.inputSchema.get("properties", {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()

                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())


## 배포된 MCP Server 테스트

원격 클라이언트를 사용하여 배포된 MCP server를 테스트해 보겠습니다:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python mcp_client_remote.py

### 원격으로 MCP Tools 호출

이제 tool을 나열할 뿐만 아니라 실제로 호출하여 전체 MCP 기능을 보여주는 향상된 클라이언트를 생성해 보겠습니다:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import sys
import os
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
# AWS SigV4 인증을 지원하는 streamable HTTP client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # 현재 boto3 session에서 AWS 자격 증명 가져오기
    session = boto3.Session()
    credentials = session.get_credentials()

    # SigV4 서명 기능을 가진 custom transport 생성
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # 모든 페이지를 가져올 때까지 반복
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    # Parameter Store에서 Agent ARN 가져오기
    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    # URL encoding: ARN의 특수 문자를 URL 안전 형식으로 변환
    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")

                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)

                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers", arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers", arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user", arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                print("\n✅ MCP tool testing completed!")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())


## Tool 호출 테스트

실제로 MCP tool을 호출하여 테스트해 보겠습니다:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 다음 단계

AgentCore Runtime에 MCP server를 성공적으로 배포했으므로 다음을 수행할 수 있습니다:

1. **더 많은 Tools 추가**: 추가 tool로 MCP server 확장
2. **커스텀 인증**: AWS IAM inbound 인증 구현
3. **통합**: 다른 AgentCore 서비스와 통합

## 정리 (선택 사항)

이 튜토리얼 중에 생성된 리소스를 정리하려면 다음 셀을 실행하세요:

In [ ]:
# try:
#     # Parameter Store에서 저장된 parameter 삭제
#     ssm_client.delete_parameter(Name='/mcp_server/runtime_iam/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

In [ ]:
# # AgentCore 리소스 정리 (ECR repository 포함)
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 🎉 축하합니다!

다음을 성공적으로 완료했습니다:

✅ **MCP server 생성** 커스텀 도구 포함  
✅ **로컬 테스트** MCP 클라이언트 사용  
✅ **인증 설정** Amazon Cognito 사용  
✅ **AWS에 배포** AgentCore Runtime 사용  
✅ **원격 호출** 적절한 인증 사용  
✅ **MCP 개념 학습** 및 모범 사례  

이제 MCP server가 Amazon Bedrock AgentCore Runtime에서 실행되고 프로덕션 사용 준비가 완료되었습니다!

## 요약

이 튜토리얼에서는 다음 방법을 배웠습니다:
- FastMCP를 사용하여 MCP server 구축
- AgentCore 호환성을 위한 stateless HTTP 전송 구성
- AWS IAM inbound 인증 설정
- AWS에서 MCP server 배포 및 관리
- 로컬 및 원격 테스트
- tool 호출을 위한 MCP 클라이언트 사용

배포된 MCP server는 이제 더 큰 AI 애플리케이션 및 워크플로우에 통합될 수 있습니다!